## Practical introduction to SAPT

- Calculating interaction energies $E_\text{int}$ of systems
- Decomposition of those $E_\text{int}$'s to physically relevant quantities
- Schrödinger equation: <br/>
  $\hat{H}\Psi = E\Psi$

<img src="images/dms_so2_mol.png" width="300" height="300"/>

- Supra-molecular methods: varying $\Psi$
  - calculate monomers A and B separately
  - calculate dimer AB
  - $E_\text{int} = E_\text{AB} - E_\text{A} - E_\text{B}$
  - suffers from BSSE
  - EDA is tacked "on top"

- Symmetry-adapted perturbation theory: varying $\hat{H}$
  - use polarisation expansion & exchange terms
  - provides EDA directly:
    $E_\text{int} = E_\text{exch} + E_\text{elst} + E_\text{ind} + E_\text{disp}$
  - higher order terms become hard to interpret
  - no charge transfer term  

### Useful levels of SAPT

- a good set of recommendations by [Parker et al.](https://doi.org/10.1063/1.4867135):
  - sSAPT0/jun-cc-pvDZ
  - SAPT(DFT)/aug-cc-pvDZ
  - SAPT2+/aug-cc-pvDZ
  - SAPT2+(3)$\delta$MP2/aug-cc-pvTZ
  - SAPT2+(CCD)$\delta$MP2/aug-cc-pvTZ
- Psi4 great at WFT SAPT
- Psi4 not the best at SAPT(DFT)

<img src="images/sapt_levels.jpg" />

### Running SAPT:

- molecule input:
  - **has** to have a fragmented molecule and symmetry C$_1$
  - `no_com` and `no_reorient` are helpful
  
  ```
  molecule dimer {
    0 1
    Xe 0 0 0
    --
    0 1
    Xe 0 0 4.1
    symmetry c1
    no_com
    no_reorient
  }
  ```

- execution with:
```
energy("sSAPT0/jun-cc-pvdz",
         molecule = dimer)
```

- for SAPT(DFT), you need to provide a gradient correction to the functional. See [this link](https://psicode.org/psi4manual/master/sapt.html#sapt-dft).

### My own examples:

Energy decomposition of Rg···OCS complexes:
<img src="images/ternary-1.png" width="300" />

Interaction energy surface of Ne-OCS:
<img src="images/Ne-OCS-1.png" width="300" />

### My own examples:

\[H$_2$···HCO\]$^+$ dissociation:
<img src="images/h2hco.jpg" width="300" />

### F/I-SAPT
- functional group / intramolecular SAPT
  - very powerful to decompose contributions to $E_\text{int}$ between functional groups
  - see more details [here](https://psicode.org/psi4manual/master/fisapt.html)

- Example: DMS···SO$_2$ complex:<br/>

$E_\text{int}$   |  $E_\text{exch}$ | $E_\text{elst}$  | $E_\text{disp}$| $E_\text{ind}(\text{AB})$ | $E_\text{ind}(\text{BA})$ |
:---------------:|:----------------:|:---------------:|:---------------:|:-------------------------:|:-------------------------:|
<img src="images/Eint.png" width="200" />|<img src="images/Exch.png" width="200" />|<img src="images/Elst.png" width="200" />|<img src="images/Disp.png" width="200" />|<img src="images/IndAB.png" width="200" />|<img src="images/IndBA.png" width="200" />

### Focal point methods in Psi4

- SCF energy converges quickly wrt. basis set size
- correlation energy converges comparably slowly
- Combine: 
  - $E_\text{HF}$ from a large basis set (usually extrapolated)
  - $E^\text{corr}_\text{MP2}$ from a medium basis set (usually extrapolated)
  - $\Delta E^\text{corr}_\text{CCSD(T)}$ from difference with MP2 in a smaller basis set
  - \[...\]

- Composite methods: mix and match other corrections:
  - basis set augmentation
  - core-valence correlation
  - relativistic effects
  - \[...\]

### CBS wrapper:

- supports simple basis set extrapolation syntax:
```
energy("HF/cc-pv[q5]z")
```
- supports simple $\Delta$ corrections:
```
energy("MP2/cc-pv[tq]z+D:CCSD(T)/cc-pvtz")
```

- more complicated protocols can be achieved using `cbs_metadata` argument:
```
params = [
  {"wfn": "hf", "basis": "cc-pv[q5]z", 
   "alpha": 1.63, 
   "scheme": scf_xtpl_karton2},
  {"wfn": "mp2", "basis": "cc-pv[tq]z",
   "alpha": 2.4, 
   "scheme": scf_xtpl_truhlar2},
  {"wfn": "ccsd(t)", "basis": "cc-pvtz"}
]
energy(cbs, cbs_metadata = params)
```

Full manual at https://psicode.org/psi4manual/master/cbs.html.

### Special things in Psi4's CBS wrapper:

- can do analytic gradients
- can do findif gradients and combine with analytic (automatically)
- can have "stages" with arbitrary options (frozen core, relativistic)
- can declare your own xtpl function and pass it as argument

### CBS advanced example:
#### Xe···OCS potential energy surface:

<img src="images/Xe-OCS-1.png" width=700 height=400/>

$\color{green}{\circ}$ HF/cc-pwcv\[tq5\]z-pp + MP2$^\text{corr}$/cc-pwcv\[q5\]-pp + $\Delta$(T)@(CCSD(T)-MP2)/cc-pwcv\[tq\]-pp

$\color{purple}{\circ}$ HF/cc-pwcv\[tq5\]z-pp + MP2$^\text{corr}$/cc-pwcv\[q5\]-pp + $\Delta$(T)@(CCSD(T)-MP2)/cc-pwcv\[tq\]-pp + $\delta_\text{rel}^4$@CCSD(T)/(TZP-dkh - TZP)

### Extrapolation of DFT

- new functionality, only available in Psi4
- planned for 1.5


Total DFA "recipe" energy defined as:
  $E_\text{tot} = E_\text{fctl} + \Delta E_\text{dh} + \Delta E_\text{disp} + \Delta E_\text{nl}$
  
The $E_\text{fctl}$ component can be further split to:
  $E_\text{fctl} = a_x E_x^\text{DFA} + (1 - a_x) E_x^\text{HF} + a_c E_c^\text{DFA}$
  and $\Delta E_\text{dh} = (1 - a_c) E_c^\text{MP2}$

Use numerical complete basis set data for diatomic molecules with PBE-like functionals to fit:

$\alpha = \alpha_0 + \alpha_x a_x + \alpha_c a_c$

### Extrapolation procedure

- use exponential-square root for $E_\text{fctl}$ component with $\alpha$ as above
- use power law for $\Delta E_\text{dh}$ with $\alpha = 2.4$ or $3.0$
- do not extrapolate $\Delta E_\text{disp}$ or $\Delta E_\text{nl}$

#### Results

works for DHDFAs | also with pcseg-N | and for range-separated DFAs |
:---------------:|:---------------:|:---------------:|
<img src="images/dsdblyp-1.png" width="300" />|<img src="images/pbe0dh-1.png" width="300" />|<img src="images/wb97xv-1.png" width="300" />|

### Thanks for listening!